# Reddit Fornite scrape
This notebook has code to scrape comments for one month of reddit. Original version used a Reddit API, but now I use the `pushshift` API. Pushshift typically yields ~300k comments / month for a month during the season.

In [40]:
#import praw # for direct reddit pull
import time
import re
import datetime
import requests
import importlib
from datetime import datetime
from ast import literal_eval
import json
import pandas as pd
import numpy as np
%matplotlib inline

In [4]:
import scrape_reddit_comments as src

nba_coach_df.to_csv('d:/data/sentiment_sports/')

In [11]:
%%time
league = 'fortnitebr'
year = 2018
month = 1
month_df, submissions, comments = src.get_month_pushshift(year,month,31, league)

Downloaded 35828 submissions
Made dataframe of shape (35828, 9)
Downloaded 233966 comments
Wall time: 35min 44s


In [12]:
month_df.sample()

,text,timestamp,user,flair,score,id,link_id,parent_id,source
56844,Noted!,1515617219,tha__smoothness,AR commander,-1,dshig2n,t3_7pfwcv,t1_dshc64j,comment


### Save / load
Something weird is going on where is throws a `UnicodeDecodeError` when writing to a gzipped file

In [13]:
drive = 'd'

In [14]:
month_df.to_csv(f'{drive}:/data/fortnite/{year}{month:02}-{league}-comments_submissions.tsv',
                sep = '\t', encoding = 'utf-8',index = False)

## Scrape and combine covariate data
### Skin data

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [136]:
#def scrape_skins_number():

with open('skins_list.html', 'r') as skin_file:
    skins = skin_file.read()
skins = skins.replace('</td>\n<td>', '')
soup = BeautifulSoup(skins, 'html.parser')
all_p = soup.find_all('p')
filtered_elements = [element for element in all_p  if len(str(element)) < 50 and 'nextpage' not in str(element) and len(str(element)) > 7]
skin_names, skin_source, skin_rarity = zip(*[(element.text, 
                                              str(element.next_sibling),
                                             list(element.next_elements)[4]) for element in filtered_elements])
#, element.next_sibling.next_sibling

In [139]:
skin_df = pd.DataFrame(list(zip(*[skin_names, skin_source, skin_rarity])),
                      columns = ['skin_name', 'skin_source', 'skin_rarity'])
skin_df.sample(3)

,skin_name,skin_source,skin_rarity
208,Ludwig,1500 V-Bucks,Epic
134,First Strike Specialist,1200 V-Bucks,Rare
369,Super Striker,1200 V-Bucks,Rare


### Scrape release dates

In [3]:
import pandas as pd

In [58]:
def get_skin_info(skin_name, base_url = 'https://fortnite.gamepedia.com/{}_(outfit)'):
    time.sleep(0.5)
    release_date=''
    skin_set = ''
    skin_url = base_url.format(skin_name.replace(' ', '_'))
    try:
        skin_info_df = pd.read_html(skin_url)[2]

        # get set info
        set_indices = skin_info_df.iloc[:,0] == "Set"
        if set_indices.sum() > 0:
            skin_set = skin_info_df.loc[set_indices].iloc[0,1]

        # get release date
        release_indices = set_indices = skin_info_df.iloc[:,0] == "Release Date (UTC-)"
        if release_indices.sum() > 0:
            release_date_string = skin_info_df.loc[release_indices].iloc[0,1]
            release_date = release_date_string[:release_date_string.find('201')+4]
    except:
        print('Could not find URL for skin ' + skin_name)
    return skin_set, release_date

In [38]:
skin_df = pd.read_csv('modeling_data/labeled_skin_covariates.tsv', sep='\t')

In [59]:
release_results = list(zip(*skin_df['skin_name'].apply(get_skin_info)))

Could not find URL for skin Bravo Leader
Could not find URL for skin B.R.U.T.E. Gunner
Could not find URL for skin B.R.U.T.E. Navigator
Could not find URL for skin Bachii
Could not find URL for skin Bubble Bomber


C:\Users\map22\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Could not find URL for skin Clinical Crossover
Could not find URL for skin Cloudbreaker
Could not find URL for skin Copper Wasp
Could not find URL for skin Dream Flower
Could not find URL for skin Flap Jackie
Could not find URL for skin Gage
Could not find URL for skin Galaxy Skin
Could not find URL for skin Gemini
Could not find URL for skin Grit
Could not find URL for skin IKONIK
Could not find URL for skin Ice Queen
Could not find URL for skin Mika
Could not find URL for skin NOG Ops
Could not find URL for skin P.A.N.D.A. Team Leader
Could not find URL for skin Prisoner
Could not find URL for skin Star Spangled Ranger
Could not find URL for skin Star Spangled Trooper
Could not find URL for skin Sugar Plum
Could not find URL for skin Tilted Teknique
Could not find URL for skin Tsuki
Could not find URL for skin Valykyrie
Could not find URL for skin Wingtip


In [61]:
skin_df['skin_set'] = release_results[0]
skin_df['release_date'] = release_results[1]

In [63]:
skin_df.to_csv('modeling_data/labeled_skin_covariates.tsv', sep='\t', index=False)

#### Fill in missing data
Before doing this I fixed typoes in Clinical Crosser and Valkyrie

In [5]:
skin_df = pd.read_csv('modeling_data/labeled_skin_covariates.tsv', sep='\t')

In [11]:
def get_progameguides_release(skin):
    time.sleep(0.5)
    try:
        skin = skin.replace(' ', '-')
        url = f'http://progameguides.com/fortnite-cosmetic/{skin}/'
        print('Requesting url: \n' + url)
        skin_html = requests.get(url).text
        return re.search('Release Date:<\/strong> (\d+\/\d+\/\d+)', skin_html).groups()[0]
    except:
        print('Had problem with ' + skin)
        return ''
def get_empty_release_date(row):
    if row['release_date'] == '' or row['release_date'] != row['release_date']:
        row['release_date'] = get_progameguides_release(row['skin_name'])
    return row

In [12]:
skin_df = skin_df.apply(get_empty_release_date, axis=1)

Requesting url: 
http://progameguides.com/fortnite-cosmetic/B.R.U.T.E.-Navigator/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Bachii/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Battlehawk/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Blackheart/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Blue-Striker/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Blue-Team-Leader/
Had problem with Blue-Team-Leader
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Brawler/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Brilliant-Striker/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Bubble-Bomber/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Bunker-Jonesy/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Calamity/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Carbide/
Requesting url: 
http://progameguides.com/fortnite-cosmetic/Carbon-Commando/
Had pr

In [15]:
skin_df['release_date'] = skin_df['release_date'].str.replace('/', '-')
skin_df['release_date'] = skin_df['release_date'].replace({'':None})

In [46]:
def convert_time_str(time_str):
    if not time_str:
        return None
    if re.search('[a-z]', time_str):
        return datetime.strptime(time_str, '%d-%b-%y')
    return datetime.strptime(time_str, '%m-%d-%Y')

In [47]:
skin_df['release_datetime'] = skin_df['release_date'].apply(convert_time_str)

In [51]:
skin_df.sample(5)

,skin_name,skin_source,skin_rarity,race,gender,story_skin,skin_set,release_date,release_datetime
202,Leviathan,2000 V-Bucks,Legendary,NaN,M,NaN,Space Explorers,16-Apr-18,2018-04-16
266,Plague,1500 V-Bucks,Epic,NaN,M,NaN,Grim Medicine,12-Oct-18,2018-10-12
411,The Visitor,Complete Blockbuster Challenges in Season 4,Legendary,NaN,M,True,NaN,05-01-2018,2018-05-01
265,Pillar,1200 V-Bucks,Rare,NaN,F,NaN,NaN,13-Jul-19,2019-07-13
241,Munitions Expert,1200 V-Bucks,Rare,L,F,NaN,NaN,27-Oct-17,2017-10-27


In [53]:
skin_df.to_csv('modeling_data/labeled_skin_covariates_verify.tsv', sep='\t', index=None)

Still some missing, but I'm going to go with what I got